# 7.2 异步Web框架FastAPI实战

## 📚 用途说明
本笔记本深入实践FastAPI异步Web框架，从基础概念到完整应用开发。重点掌握Pydantic数据验证、异步请求处理、流式响应等核心特性，为构建高性能LangChain Web应用奠定基础。

### 🎯 学习目标
- 掌握FastAPI的核心特性和最佳实践
- 深入理解Pydantic数据验证和类型提示
- 实现异步请求处理和流式响应
- 构建完整的FastAPI应用架构
- 为LangChain集成设计API接口

### ⚡ 前置要求
- HTTP协议基础和Web框架概念
- 异步编程基础（协程、事件循环）
- Python类型提示和装饰器
- JSON数据格式和RESTful API概念

### 🔄 LangChain关联
FastAPI是构建LangChain Web应用的最佳选择：
- 异步处理：支持并发AI请求和响应
- 流式响应：实现实时对话和生成内容
- 类型安全：确保AI数据结构的正确性
- 自动文档：为LangChain API生成完整文档

### 📦 依赖安装
```bash
pip install fastapi uvicorn pydantic python-multipart
pip install aiofiles  # 用于异步文件操作
```

In [ ]:
# 导入FastAPI相关依赖
import asyncio
import json
import time
import uuid
from typing import Dict, List, Optional, Any, AsyncGenerator
from datetime import datetime
from dataclasses import dataclass
from contextlib import asynccontextmanager

# FastAPI核心导入
try:
    from fastapi import FastAPI, HTTPException, Depends, BackgroundTasks, Request, Response
    from fastapi.responses import StreamingResponse, JSONResponse
    from fastapi.middleware.cors import CORSMiddleware
    from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
    from pydantic import BaseModel, Field, validator, EmailStr
    from pydantic.generics import GenericModel
    from typing import Generic, TypeVar
    
    FASTAPI_AVAILABLE = True
    print("✅ FastAPI依赖导入成功")
except ImportError as e:
    FASTAPI_AVAILABLE = False
    print(f"❌ FastAPI依赖缺失: {e}")
    print("请运行: pip install fastapi uvicorn pydantic")

print("\n🚀 异步Web框架FastAPI实战")
print("=" * 50)
print(f"📝 学习目标: 掌握FastAPI实战开发")
print(f"⚡ 异步特性: 利用FastAPI构建高性能Web应用")

## 1. FastAPI基础入门

### 1.1 创建FastAPI应用实例

In [ ]:
print(f"\n🏗️ 1. FastAPI基础入门:")

if FASTAPI_AVAILABLE:
    # 创建FastAPI应用实例
    app = FastAPI(
        title="LangChain FastAPI Demo",
        description="FastAPI实战演示应用",
        version="1.0.0",
        docs_url="/docs",
        redoc_url="/redoc"
    )
    
    print(f"   ✅ FastAPI应用创建成功")
    print(f"      - 应用标题: {app.title}")
    print(f"      - 版本: {app.version}")
    print(f"      - API文档: /docs")
    print(f"      - ReDoc文档: /redoc")
    
    # 基础路由示例
    @app.get("/")
    async def root():
        """根路径处理函数"""
        return {
            "message": "Welcome to FastAPI with LangChain",
            "timestamp": datetime.now().isoformat(),
            "version": app.version
        }
    
    @app.get("/health")
    async def health_check():
        """健康检查端点"""
        return {
            "status": "healthy",
            "timestamp": datetime.now().isoformat(),
            "service": "fastapi-langchain-demo"
        }
    
    print(f"   ✅ 基础路由创建完成")
    print(f"      - GET /: 欢迎页面")
    print(f"      - GET /health: 健康检查")
else:
    print(f"   ❌ FastAPI不可用，跳过示例创建")

### 1.2 应用生命周期管理

In [ ]:
print(f"\n🔄 2. 应用生命周期管理:")

if FASTAPI_AVAILABLE:
    # 应用状态管理
    @asynccontextmanager
    async def lifespan(app: FastAPI):
        """应用生命周期管理"""
        # 启动时执行
        print(f"      🚀 FastAPI应用启动中...")
        
        # 初始化资源
        app.state.start_time = datetime.now()
        app.state.request_count = 0
        app.state.langchain_models = ["gpt-3.5-turbo", "gpt-4", "claude-3"]
        
        print(f"      ✅ 应用初始化完成")
        print(f"      📊 可用模型: {len(app.state.langchain_models)}个")
        
        yield
        
        # 关闭时执行
        print(f"      🛑 FastAPI应用关闭中...")
        # 清理资源
        print(f"      ✅ 资源清理完成")
    
    # 重新创建带生命周期的应用
    app = FastAPI(
        title="LangChain FastAPI Demo",
        description="FastAPI实战演示应用",
        version="1.0.0",
        lifespan=lifespan
    )
    
    @app.get("/")
    async def root():
        """根路径处理函数"""
        return {
            "message": "Welcome to FastAPI with LangChain",
            "timestamp": datetime.now().isoformat(),
            "version": app.version
        }
    
    @app.get("/health")
    async def health_check():
        """健康检查端点"""
        uptime = datetime.now() - app.state.start_time
        return {
            "status": "healthy",
            "timestamp": datetime.now().isoformat(),
            "service": "fastapi-langchain-demo",
            "uptime_seconds": uptime.total_seconds(),
            "request_count": app.state.request_count
        }
    
    print(f"   ✅ 生命周期管理配置完成")
    print(f"      - 启动时: 初始化应用状态和资源")
    print(f"      - 运行时: 维护应用状态")
    print(f"      - 关闭时: 清理资源和连接")
else:
    print(f"   ❌ FastAPI不可用，跳过生命周期配置")

## 2. Pydantic数据验证

### 2.1 基础数据模型定义

In [ ]:
print(f"\n📋 3. Pydantic数据验证:")

if FASTAPI_AVAILABLE:
    # 用户数据模型
    class UserBase(BaseModel):
        """用户基础模型"""
        username: str = Field(..., min_length=3, max_length=50, description="用户名")
        email: EmailStr = Field(..., description="邮箱地址")
        full_name: Optional[str] = Field(None, max_length=100, description="全名")
        
        @validator('username')
        def username_alphanumeric(cls, v):
            if not v.isalnum():
                raise ValueError('用户名只能包含字母和数字')
            return v.lower()
    
    class UserCreate(UserBase):
        """用户创建模型"""
        password: str = Field(..., min_length=6, max_length=100, description="密码")
        confirm_password: str = Field(..., description="确认密码")
        
        @validator('confirm_password')
        def passwords_match(cls, v, values):
            if 'password' in values and v != values['password']:
                raise ValueError('密码确认不匹配')
            return v
    
    class UserResponse(UserBase):
        """用户响应模型"""
        id: int = Field(..., description="用户ID")
        created_at: datetime = Field(..., description="创建时间")
        is_active: bool = Field(True, description="是否激活")
        
        class Config:
            from_attributes = True
    
    print(f"   ✅ 用户数据模型创建完成")
    print(f"      - UserBase: 基础用户信息")
    print(f"      - UserCreate: 用户创建（含密码验证）")
    print(f"      - UserResponse: 用户响应（不含密码）")
    
    # 测试数据验证
    print(f"\n   🧪 测试数据验证:")
    
    try:
        # 有效数据
        valid_user = UserCreate(
            username="TestUser123",
            email="test@example.com",
            full_name="测试用户",
            password="secure123",
            confirm_password="secure123"
        )
        print(f"      ✅ 有效数据验证通过: {valid_user.username}")
    except Exception as e:
        print(f"      ❌ 有效数据验证失败: {e}")
    
    try:
        # 无效数据（密码不匹配）
        invalid_user = UserCreate(
            username="TestUser123",
            email="test@example.com",
            password="secure123",
            confirm_password="different"
        )
        print(f"      ❌ 无效数据应该失败但通过了")
    except Exception as e:
        print(f"      ✅ 无效数据正确被拒绝: {e}")
else:
    print(f"   ❌ FastAPI不可用，跳过数据模型创建")

### 2.2 LangChain相关数据模型

In [ ]:
print(f"\n🤖 4. LangChain相关数据模型:")

if FASTAPI_AVAILABLE:
    # 聊天消息模型
    class MessageRole(str):
        USER = "user"
        ASSISTANT = "assistant"
        SYSTEM = "system"
    
    class ChatMessage(BaseModel):
        """聊天消息模型"""
        role: str = Field(..., description="消息角色")
        content: str = Field(..., min_length=1, max_length=4000, description="消息内容")
        timestamp: datetime = Field(default_factory=datetime.now, description="时间戳")
        
        @validator('role')
        def validate_role(cls, v):
            if v not in [MessageRole.USER, MessageRole.ASSISTANT, MessageRole.SYSTEM]:
                raise ValueError(f'角色必须是: {MessageRole.USER}, {MessageRole.ASSISTANT}, {MessageRole.SYSTEM}')
            return v
    
    # 聊天请求模型
    class ChatRequest(BaseModel):
        """聊天请求模型"""
        messages: List[ChatMessage] = Field(..., min_items=1, description="消息列表")
        model: str = Field("gpt-3.5-turbo", description="使用的模型")
        temperature: float = Field(0.7, ge=0.0, le=2.0, description="温度参数")
        max_tokens: Optional[int] = Field(None, ge=1, le=4000, description="最大token数")
        stream: bool = Field(False, description="是否流式响应")
        
        @validator('messages')
        def validate_messages(cls, v):
            if not v:
                raise ValueError('消息列表不能为空')
            # 检查最后一条消息是否来自用户
            if v[-1].role != MessageRole.USER:
                raise ValueError('最后一条消息必须来自用户')
            return v
    
    # 聊天响应模型
    class ChatResponse(BaseModel):
        """聊天响应模型"""
        id: str = Field(default_factory=lambda: str(uuid.uuid4()), description="响应ID")
        message: ChatMessage = Field(..., description="AI回复消息")
        model: str = Field(..., description="使用的模型")
        usage: Dict[str, int] = Field(..., description="token使用情况")
        created_at: datetime = Field(default_factory=datetime.now, description="创建时间")
    
    # RAG查询模型
    class RAGQuery(BaseModel):
        """RAG查询模型"""
        query: str = Field(..., min_length=1, max_length=1000, description="查询问题")
        collection: str = Field("default", description="文档集合")
        top_k: int = Field(5, ge=1, le=20, description="返回文档数量")
        similarity_threshold: float = Field(0.7, ge=0.0, le=1.0, description="相似度阈值")
    
    class Document(BaseModel):
        """文档模型"""
        id: str = Field(..., description="文档ID")
        content: str = Field(..., description="文档内容")
        metadata: Dict[str, Any] = Field(default_factory=dict, description="元数据")
        score: float = Field(..., description="相似度分数")
    
    class RAGResponse(BaseModel):
        """RAG响应模型"""
        query: str = Field(..., description="原始查询")
        answer: str = Field(..., description="生成的答案")
        documents: List[Document] = Field(..., description="检索到的文档")
        processing_time: float = Field(..., description="处理时间（秒）")
    
    print(f"   ✅ LangChain数据模型创建完成")
    print(f"      - ChatMessage: 聊天消息结构")
    print(f"      - ChatRequest: 聊天请求参数")
    print(f"      - ChatResponse: 聊天响应格式")
    print(f"      - RAGQuery: 检索查询参数")
    print(f"      - Document: 文档结构")
    print(f"      - RAGResponse: RAG响应格式")
    
    # 测试LangChain模型
    print(f"\n   🧪 测试LangChain数据模型:")
    
    try:
        # 创建聊天请求
        chat_request = ChatRequest(
            messages=[
                ChatMessage(role=MessageRole.USER, content="你好，请介绍一下Python")
            ],
            model="gpt-3.5-turbo",
            temperature=0.7,
            stream=False
        )
        print(f"      ✅ 聊天请求验证通过: {len(chat_request.messages)}条消息")
    except Exception as e:
        print(f"      ❌ 聊天请求验证失败: {e}")
    
    try:
        # 创建RAG查询
        rag_query = RAGQuery(
            query="Python异步编程的最佳实践",
            collection="programming_docs",
            top_k=5,
            similarity_threshold=0.8
        )
        print(f"      ✅ RAG查询验证通过: {rag_query.query}")
    except Exception as e:
        print(f"      ❌ RAG查询验证失败: {e}")
else:
    print(f"   ❌ FastAPI不可用，跳过LangChain模型创建")

## 3. 异步请求处理

### 3.1 异步路由和依赖注入

In [ ]:
print(f"\n⚡ 5. 异步请求处理:")

if FASTAPI_AVAILABLE:
    # 模拟数据库
    class MockDatabase:
        """模拟数据库类"""
        def __init__(self):
            self.users = {}
            self.next_id = 1
        
        async def create_user(self, user_data: dict) -> dict:
            """异步创建用户"""
            await asyncio.sleep(0.01)  # 模拟数据库延迟
            user_id = self.next_id
            self.next_id += 1
            
            user = {
                "id": user_id,
                **user_data,
                "created_at": datetime.now(),
                "is_active": True
            }
            self.users[user_id] = user
            return user
        
        async def get_user(self, user_id: int) -> Optional[dict]:
            """异步获取用户"""
            await asyncio.sleep(0.01)  # 模拟数据库延迟
            return self.users.get(user_id)
        
        async def get_all_users(self) -> List[dict]:
            """异步获取所有用户"""
            await asyncio.sleep(0.01)  # 模拟数据库延迟
            return list(self.users.values())
    
    # 创建数据库实例
    db = MockDatabase()
    
    # 依赖注入函数
    async def get_database() -> MockDatabase:
        """获取数据库依赖"""
        return db
    
    # 用户路由
    @app.post("/users", response_model=UserResponse)
    async def create_user(
        user: UserCreate,
        database: MockDatabase = Depends(get_database)
    ):
        """创建用户"""
        # 转换为数据库格式
        user_data = user.dict(exclude={"confirm_password"})
        
        # 异步创建用户
        created_user = await database.create_user(user_data)
        
        return UserResponse(**created_user)
    
    @app.get("/users", response_model=List[UserResponse])
    async def get_users(
        database: MockDatabase = Depends(get_database)
    ):
        """获取所有用户"""
        users = await database.get_all_users()
        return [UserResponse(**user) for user in users]
    
    @app.get("/users/{user_id}", response_model=UserResponse)
    async def get_user(
        user_id: int,
        database: MockDatabase = Depends(get_database)
    ):
        """获取指定用户"""
        user = await database.get_user(user_id)
        if not user:
            raise HTTPException(status_code=404, detail="用户不存在")
        return UserResponse(**user)
    
    print(f"   ✅ 异步用户路由创建完成")
    print(f"      - POST /users: 创建用户（异步数据库操作）")
    print(f"      - GET /users: 获取所有用户")
    print(f"      - GET /users/{{id}}: 获取指定用户")
    print(f"      - 依赖注入: 自动注入数据库实例")
else:
    print(f"   ❌ FastAPI不可用，跳过异步路由创建")

### 3.2 异步中间件和请求处理

In [ ]:
print(f"\n🔗 6. 异步中间件和请求处理:")

if FASTAPI_AVAILABLE:
    # 请求计数中间件
    @app.middleware("http")
    async def request_counter_middleware(request: Request, call_next):
        """请求计数中间件"""
        start_time = time.time()
        
        # 增加请求计数
        app.state.request_count += 1
        
        # 处理请求
        response = await call_next(request)
        
        # 计算处理时间
        process_time = time.time() - start_time
        
        # 添加响应头
        response.headers["X-Process-Time"] = str(process_time)
        response.headers["X-Request-Count"] = str(app.state.request_count)
        
        return response
    
    # CORS中间件
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )
    
    # 全局异常处理
    @app.exception_handler(ValueError)
    async def validation_exception_handler(request: Request, exc: ValueError):
        """数据验证异常处理"""
        return JSONResponse(
            status_code=422,
            content={
                "error": "数据验证失败",
                "detail": str(exc),
                "timestamp": datetime.now().isoformat()
            }
        )
    
    @app.exception_handler(Exception)
    async def general_exception_handler(request: Request, exc: Exception):
        """通用异常处理"""
        return JSONResponse(
            status_code=500,
            content={
                "error": "服务器内部错误",
                "detail": "请求处理过程中发生错误",
                "timestamp": datetime.now().isoformat()
            }
        )
    
    print(f"   ✅ 中间件和异常处理配置完成")
    print(f"      - 请求计数中间件: 统计请求次数和处理时间")
    print(f"      - CORS中间件: 支持跨域请求")
    print(f"      - 全局异常处理: 统一的错误响应格式")
else:
    print(f"   ❌ FastAPI不可用，跳过中间件配置")

## 4. 流式响应实现

### 4.1 Server-Sent Events (SSE)

In [ ]:
print(f"\n🌊 7. 流式响应实现:")

if FASTAPI_AVAILABLE:
    # 模拟LangChain流式响应
    async def simulate_langchain_stream(message: str) -> AsyncGenerator[str, None]:
        """模拟LangChain流式响应生成器"""
        # 模拟AI思考过程
        await asyncio.sleep(0.5)
        
        # 模拟逐字生成
        words = message.split()
        response = f"关于'{message}'的回答："
        
        for i, word in enumerate(words):
            chunk = f"{word} "
            response += chunk
            
            # 发送SSE格式的数据
            data = {
                "id": str(uuid.uuid4()),
                "content": chunk,
                "finished": i == len(words) - 1,
                "timestamp": datetime.now().isoformat()
            }
            
            yield f"data: {json.dumps(data, ensure_ascii=False)}\n\n"
            await asyncio.sleep(0.1)  # 模拟生成延迟
    
    @app.post("/chat/stream")
    async def chat_stream(request: ChatRequest):
        """流式聊天接口"""
        if not request.stream:
            raise HTTPException(status_code=400, detail="此接口仅支持流式响应")
        
        # 获取最后一条用户消息
        user_message = request.messages[-1].content
        
        # 返回流式响应
        return StreamingResponse(
            simulate_langchain_stream(user_message),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "X-Accel-Buffering": "no"  # 禁用Nginx缓冲
            }
        )
    
    # 普通聊天接口（非流式）
    async def simulate_langchain_response(message: str) -> ChatResponse:
        """模拟LangChain响应"""
        await asyncio.sleep(0.5)  # 模拟AI处理时间
        
        ai_message = ChatMessage(
            role=MessageRole.ASSISTANT,
            content=f"这是对'{message}'的AI回答。我理解了您的问题，让我为您提供详细的解答...",
            timestamp=datetime.now()
        )
        
        return ChatResponse(
            message=ai_message,
            model="gpt-3.5-turbo",
            usage={
                "prompt_tokens": len(message.split()),
                "completion_tokens": 50,
                "total_tokens": len(message.split()) + 50
            }
        )
    
    @app.post("/chat", response_model=ChatResponse)
    async def chat(request: ChatRequest):
        """普通聊天接口"""
        if request.stream:
            raise HTTPException(status_code=400, detail="流式请求请使用 /chat/stream 接口")
        
        # 获取最后一条用户消息
        user_message = request.messages[-1].content
        
        # 生成AI响应
        response = await simulate_langchain_response(user_message)
        response.model = request.model
        
        return response
    
    print(f"   ✅ 流式响应接口创建完成")
    print(f"      - POST /chat: 普通聊天接口")
    print(f"      - POST /chat/stream: 流式聊天接口（SSE）")
    print(f"      - 支持逐字生成和实时传输")
else:
    print(f"   ❌ FastAPI不可用，跳过流式响应创建")

### 4.2 文件流式下载

In [ ]:
print(f"\n📁 8. 文件流式下载:")

if FASTAPI_AVAILABLE:
    # 模拟大文件生成
    async def generate_large_file(size_mb: int = 1) -> AsyncGenerator[bytes, None]:
        """生成大文件的异步生成器"""
        chunk_size = 8192  # 8KB chunks
        total_chunks = (size_mb * 1024 * 1024) // chunk_size
        
        for i in range(int(total_chunks)):
            # 生成数据块
            chunk = f"这是第{i+1}个数据块\n".encode('utf-8')
            chunk += b' ' * (chunk_size - len(chunk))  # 填充到指定大小
            
            yield chunk
            
            # 模拟I/O延迟
            if i % 100 == 0:
                await asyncio.sleep(0.001)
    
    @app.get("/download/file")
    async def download_file(size_mb: int = 1):
        """流式文件下载"""
        if size_mb < 1 or size_mb > 100:
            raise HTTPException(status_code=400, detail="文件大小必须在1-100MB之间")
        
        return StreamingResponse(
            generate_large_file(size_mb),
            media_type="application/octet-stream",
            headers={
                "Content-Disposition": f"attachment; filename=large_file_{size_mb}mb.dat",
                "Content-Length": str(size_mb * 1024 * 1024)
            }
        )
    
    # CSV数据流式导出
    async def generate_csv_data() -> AsyncGenerator[str, None]:
        """生成CSV数据的异步生成器"""
        # CSV头部
        yield "ID,Name,Email,CreatedAt\n"
        
        # 生成数据行
        for i in range(1, 1001):
            await asyncio.sleep(0.001)  # 模拟数据库查询
            
            row = f"{i},User{i},user{i}@example.com,{datetime.now().isoformat()}\n"
            yield row
    
    @app.get("/export/csv")
    async def export_csv():
        """CSV数据流式导出"""
        return StreamingResponse(
            generate_csv_data(),
            media_type="text/csv",
            headers={
                "Content-Disposition": "attachment; filename=users.csv",
                "Cache-Control": "no-cache"
            }
        )
    
    print(f"   ✅ 文件流式下载接口创建完成")
    print(f"      - GET /download/file: 大文件流式下载")
    print(f"      - GET /export/csv: CSV数据流式导出")
    print(f"      - 支持内存高效的大文件传输")
else:
    print(f"   ❌ FastAPI不可用，跳过文件流式下载创建")

## 5. 完整FastAPI应用示例

### 5.1 LangChain API服务集成

In [ ]:
print(f"\n🤖 9. 完整FastAPI应用示例:")

if FASTAPI_AVAILABLE:
    # LangChain服务模拟
    class LangChainService:
        """LangChain服务模拟类"""
        def __init__(self):
            self.models = ["gpt-3.5-turbo", "gpt-4", "claude-3"]
            self.conversations = {}  # 模拟会话存储
        
        async def chat_completion(self, request: ChatRequest) -> ChatResponse:
            """聊天完成"""
            await asyncio.sleep(0.5)  # 模拟API调用
            
            user_message = request.messages[-1].content
            
            ai_message = ChatMessage(
                role=MessageRole.ASSISTANT,
                content=f"基于{request.model}模型，我对'{user_message}'的回答：这是一个详细的AI响应...",
                timestamp=datetime.now()
            )
            
            return ChatResponse(
                message=ai_message,
                model=request.model,
                usage={
                    "prompt_tokens": len(user_message),
                    "completion_tokens": 100,
                    "total_tokens": len(user_message) + 100
                }
            )
        
        async def rag_query(self, query: RAGQuery) -> RAGResponse:
            """RAG查询"""
            start_time = time.time()
            
            # 模拟文档检索
            await asyncio.sleep(0.2)
            
            documents = [
                Document(
                    id=f"doc_{i}",
                    content=f"关于{query.query}的相关文档内容{i}...",
                    metadata={"source": f"source_{i}", "page": i},
                    score=0.9 - (i * 0.1)
                ) for i in range(min(query.top_k, 5))
            ]
            
            # 模拟答案生成
            await asyncio.sleep(0.3)
            
            answer = f"基于检索到的{len(documents)}个文档，关于'{query.query}'的答案是：..."
            
            processing_time = time.time() - start_time
            
            return RAGResponse(
                query=query.query,
                answer=answer,
                documents=documents,
                processing_time=processing_time
            )
    
    # 创建LangChain服务实例
    langchain_service = LangChainService()
    
    # 依赖注入
    async def get_langchain_service() -> LangChainService:
        return langchain_service
    
    # LangChain API路由
    @app.post("/api/v1/chat", response_model=ChatResponse)
    async def api_chat(
        request: ChatRequest,
        service: LangChainService = Depends(get_langchain_service)
    ):
        """LangChain聊天API"""
        if request.model not in service.models:
            raise HTTPException(
                status_code=400,
                detail=f"不支持的模型: {request.model}"
            )
        
        return await service.chat_completion(request)
    
    @app.post("/api/v1/rag", response_model=RAGResponse)
    async def api_rag(
        query: RAGQuery,
        service: LangChainService = Depends(get_langchain_service)
    ):
        """LangChain RAG查询API"""
        return await service.rag_query(query)
    
    @app.get("/api/v1/models")
    async def get_models(
        service: LangChainService = Depends(get_langchain_service)
    ):
        """获取可用模型列表"""
        return {
            "models": service.models,
            "count": len(service.models),
            "timestamp": datetime.now().isoformat()
        }
    
    # 批量处理API
    @app.post("/api/v1/chat/batch")
    async def batch_chat(
        requests: List[ChatRequest],
        service: LangChainService = Depends(get_langchain_service)
    ):
        """批量聊天处理"""
        if len(requests) > 10:
            raise HTTPException(
                status_code=400,
                detail="批量请求最多支持10个"
            )
        
        # 并发处理多个请求
        tasks = [service.chat_completion(req) for req in requests]
        results = await asyncio.gather(*tasks)
        
        return {
            "results": results,
            "count": len(results),
            "processing_time": sum(r.usage.get("total_tokens", 0) for r in results)
        }
    
    print(f"   ✅ LangChain API服务集成完成")
    print(f"      - POST /api/v1/chat: 聊天完成API")
    print(f"      - POST /api/v1/rag: RAG查询API")
    print(f"      - GET /api/v1/models: 模型列表API")
    print(f"      - POST /api/v1/chat/batch: 批量聊天API")
    print(f"      - 异步并发处理支持")
else:
    print(f"   ❌ FastAPI不可用，跳过LangChain API集成")

### 5.2 应用启动和测试

In [ ]:
print(f"\n🚀 10. 应用启动和测试:")

if FASTAPI_AVAILABLE:
    # 应用信息展示
    def print_app_info():
        """打印应用信息"""
        print(f"   📋 FastAPI应用信息:")
        print(f"      标题: {app.title}")
        print(f"      版本: {app.version}")
        print(f"      描述: {app.description}")
        
        print(f"\n   🛣️ 可用路由:")
        for route in app.routes:
            if hasattr(route, 'methods') and hasattr(route, 'path'):
                methods = ', '.join(route.methods)
                print(f"      {methods:<8} {route.path}")
        
        print(f"\n   📚 API文档:")
        print(f"      Swagger UI: http://localhost:8000/docs")
        print(f"      ReDoc: http://localhost:8000/redoc")
        print(f"      OpenAPI JSON: http://localhost:8000/openapi.json")
    
    print_app_info()
    
    # 启动说明
    print(f"\n   🔧 启动FastAPI应用:")
    print(f"      方法1: 命令行启动")
    print(f"        uvicorn main:app --host 0.0.0.0 --port 8000 --reload")
    print(f"      ")
    print(f"      方法2: Python代码启动")
    print(f"        import uvicorn")
    print(f"        uvicorn.run(app, host='0.0.0.0', port=8000)")
    print(f"      ")
    print(f"      方法3: 开发模式启动")
    print(f"        uvicorn main:app --reload --log-level debug")
    
    # 测试用例
    async def test_api_endpoints():
        """测试API端点"""
        from fastapi.testclient import TestClient
        
        client = TestClient(app)
        
        print(f"\n   🧪 API端点测试:")
        
        # 测试基础端点
        response = client.get("/")
        print(f"      GET /: {response.status_code} - {response.json().get('message', '')}")
        
        response = client.get("/health")
        print(f"      GET /health: {response.status_code} - {response.json().get('status', '')}")
        
        # 测试用户API
        user_data = {
            "username": "testuser123",
            "email": "test@example.com",
            "password": "secure123",
            "confirm_password": "secure123"
        }
        
        response = client.post("/users", json=user_data)
        if response.status_code == 200:
            user = response.json()
            print(f"      POST /users: {response.status_code} - 创建用户 {user['username']}")
            
            response = client.get("/users")
            users = response.json()
            print(f"      GET /users: {response.status_code} - 获取{len(users)}个用户")
        else:
            print(f"      POST /users: {response.status_code} - {response.text}")
        
        # 测试LangChain API
        chat_data = {
            "messages": [
                {
                    "role": "user",
                    "content": "你好，请介绍一下FastAPI"
                }
            ],
            "model": "gpt-3.5-turbo",
            "temperature": 0.7,
            "stream": False
        }
        
        response = client.post("/api/v1/chat", json=chat_data)
        if response.status_code == 200:
            chat_response = response.json()
            print(f"      POST /api/v1/chat: {response.status_code} - AI响应生成成功")
        else:
            print(f"      POST /api/v1/chat: {response.status_code} - {response.text}")
        
        response = client.get("/api/v1/models")
        if response.status_code == 200:
            models = response.json()
            print(f"      GET /api/v1/models: {response.status_code} - {models['count']}个可用模型")
    
    # 运行测试
    try:
        await test_api_endpoints()
    except Exception as e:
        print(f"      ⚠️ 测试需要安装 httpx: pip install httpx")
        print(f"      错误详情: {e}")
    
    print(f"\n   ✅ FastAPI应用创建和测试完成")
    print(f"      🎯 核心特性: 异步处理、数据验证、流式响应、自动文档")
    print(f"      🚀 性能优势: 高并发、低延迟、类型安全")
    print(f"      🤖 LangChain集成: 完美的AI Web应用开发框架")
else:
    print(f"   ❌ FastAPI不可用，无法进行应用测试")

## 学习总结

### 📚 知识点回顾

In [ ]:
print(f"\n📚 学习总结:")
print("=" * 50)

class FastAPILearningSummary:
    """FastAPI学习内容总结"""
    
    topics_covered = [
        "FastAPI应用创建和基础配置",
        "应用生命周期管理和资源初始化",
        "Pydantic数据模型和验证机制",
        "LangChain专用数据结构设计",
        "异步路由和依赖注入模式",
        "中间件系统和全局异常处理",
        "流式响应和Server-Sent Events",
        "文件流式传输和CSV导出",
        "LangChain API服务集成",
        "批量处理和并发请求优化"
    ]
    
    practical_skills = [
        "能够创建完整的FastAPI应用架构",
        "掌握Pydantic数据验证和类型提示",
        "实现异步请求处理和依赖注入",
        "构建流式响应和实时通信接口",
        "设计LangChain集成的API服务",
        "配置中间件和全局异常处理",
        "实现批量处理和性能优化",
        "生成自动API文档和测试用例"
    ]
    
    langchain_integration_patterns = [
        "聊天API设计（同步/异步/流式）",
        "RAG查询接口和文档检索",
        "模型管理和配置API",
        "批量AI请求并发处理",
        "实时对话和SSE流式传输",
        "AI工作流和任务调度",
        "错误处理和重试机制",
        "性能监控和日志记录"
    ]
    
    @classmethod
    def print_summary(cls):
        """打印学习总结"""
        print(f"✅ 已掌握知识点:")
        for i, topic in enumerate(cls.topics_covered, 1):
            print(f"   {i}. {topic}")
        
        print(f"\n🎯 实践技能获得:")
        for i, skill in enumerate(cls.practical_skills, 1):
            print(f"   {i}. {skill}")
        
        print(f"\n🤖 LangChain集成模式:")
        for i, pattern in enumerate(cls.langchain_integration_patterns, 1):
            print(f"   {i}. {pattern}")
    
    @classmethod
    def best_practices(cls):
        """最佳实践总结"""
        print(f"\n💡 FastAPI最佳实践:")
        print(f"   1. 使用Pydantic模型进行严格的数据验证")
        print(f"   2. 利用异步特性处理I/O密集型操作")
        print(f"   3. 通过依赖注入实现代码解耦")
        print(f"   4. 使用中间件处理横切关注点")
        print(f"   5. 实现流式响应提升用户体验")
        print(f"   6. 配置全局异常处理统一错误格式")
        print(f"   7. 利用自动文档简化API维护")
        print(f"   8. 设计可扩展的LangChain集成架构")
    
    @classmethod
    def next_steps(cls):
        """下一步学习计划"""
        print(f"\n🚀 下一步学习计划:")
        print(f"   1. 深入学习RESTful API设计原则")
        print(f"   2. 掌握WebSocket实时通信技术")
        print(f"   3. 学习异步数据库操作和ORM")
        print(f"   4. 了解Web安全和身份认证机制")
        print(f"   5. 掌握API文档生成和测试自动化")
        print(f"   6. 学习微服务架构设计模式")
        print(f"   7. 了解容器化部署和运维")
        print(f"   8. 构建完整的LangChain生产应用")

# 打印学习总结
FastAPILearningSummary.print_summary()
FastAPILearningSummary.best_practices()
FastAPILearningSummary.next_steps()

print(f"\n🎉 异步Web框架FastAPI实战 学习完成!")
print(f"🚀 已掌握构建高性能LangChain Web应用的核心技能")
print(f"⚡ 异步编程和FastAPI的完美结合为AI应用开发提供了强大基础")
print(f"🔗 为后续的RESTful API设计和WebSocket实时通信做好了准备")